In [4]:
import pandas as pd
import pandas.io.sql as psql
import os
import sys
from datetime import date
current_date = date.today()
#%load_ext autotime
import re
import csv
import pandas as pd
import numpy as np
import os
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
#from matplotlib_venn import venn2, venn3
import warnings
import time
import datetime as dt
import itertools
from more_itertools import unique_everseen
warnings.filterwarnings("ignore")
#import pandasql as ps
import seaborn as sns
import json
from ast import literal_eval
import ast

In [126]:
df5 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Mayuresh\APOE33\apoe33_notes_1.csv")

In [127]:
df5_matches = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\1\apoe_1_matches.csv")

In [128]:
unique_ids = df5_matches["PatientID"].unique()
unique_ids = list(unique_ids)
len(unique_ids)

2890

In [15]:
regex = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\Sentence-Analysis\EDA\Getting_Data\keywords.csv")
regex = regex["REGEX"].to_list()

In [99]:
print(df5_matches.columns)

Index(['Unnamed: 0', 'EMPI', 'MRN', 'PatientID', 'PatientEncounterID',
       'ContactDTS', 'EncounterTypeDSC', 'NoteID', 'InpatientNoteTypeDSC',
       'buckets', 'NoteCSNID', 'LineNBR', 'NoteTXT', 'regex_location',
       'regex_sent', 'regex_match', 'match_count'],
      dtype='object')


In [132]:
cols =  ["patient_id", "encounters_with_keywords",'dementia', 'cognition',"\'cognition", 'memory', "memory\'", 'cdr', 'mmse', 'moca', 'alzheimer', 'cognitive impairment','cognitiveimpairment', 'mci', 'cerebellar', 'neurocognitive', 'lewy', "pick's", 'cortical', 'corticobasal', 'cerebral', 'cerebrovascular', 'amnesia']
summary_stats  = pd.DataFrame(pd.np.empty((len(unique_ids), len(cols))) * pd.np.nan) 
summary_stats.columns = cols
summary_stats["patient_id"] = unique_ids
summary_stats

,patient_id,encounters_with_keywords,dementia,cognition,'cognition,memory,memory',cdr,mmse,moca,...,mci,cerebellar,neurocognitive,lewy,pick's,cortical,corticobasal,cerebral,cerebrovascular,amnesia
0,Z6352398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Z6352614,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Z6352700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Z6352922,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Z6352946,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2885,Z8712370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2886,Z8713474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2887,Z8713489,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2888,Z8713893,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [146]:
for i in tqdm(range(len(unique_ids))):
    keyword_list = df5_matches[df5_matches['PatientID'] == unique_ids[i]]['regex_match'].values
    
    match_tuples = []
    for j in range(len(keyword_list)):
        sub = "\', \'"
        x = keyword_list[j][2:len(keyword_list[j])-2]
        if sub in x:
            x = x.split("\', \'")
        else:
            x = x.split("\", \"")
        match_tuples.append((x[0], 0))

    #print(match_tuples)

    freq = {"dementia":0
        ,"cognition":0
        ,"\'cognition":0
        ,"memory":0
        ,"memory\'":0
        ,"cdr":0
        ,"mmse":0
        ,"moca":0
        ,"alzheimer":0
        ,'cognitive impairment':0
        , "cognitiveimpairment":0
        ,"mci":0
        ,"cerebellar":0
        ,"neurocognitive":0
        ,"lewy":0
        ,"pick's":0
        ,"cortical":0
        ,"corticobasal":0
        ,"cerebral":0
        ,"cerebrovascular":0
        ,"amnesia":0
    }

    for k, v in match_tuples:   
        if (k != ''):
            freq.update({str(k.lower()):int(freq[k.lower()] + 1)})

    #print(freq, i, "\n")

    summary_stats["encounters_with_keywords"][i] = len(df5_matches[df5_matches["PatientID"] == unique_ids[i]])

    for k, v in freq.items():
        summary_stats[str(k.lower())][i] = int(v)
        #print(k, v)

100%|██████████████████████████████████████████████████████████████████████████████| 2890/2890 [02:35<00:00, 18.56it/s]


In [152]:
keyword_list = df5_matches[df5_matches['PatientID'] == unique_ids[436]]['regex_match'].values

match_tuples = []
for j in range(len(keyword_list)):
    sub = "\', \'"
    x = keyword_list[j][2:len(keyword_list[j])-2]
    #print("before: ",x)
    if sub in x:
        x = x.split("\', \'")
    else:
        x = x.split("\", \"")
    #print("after: ", x)
    #
    #print(x)
    match_tuples.append((x[0], 0))

#print(match_tuples)

freq = {"dementia":0
    ,"cognition":0
    ,"\'cognition":0
    ,"memory":0
    ,"memory\'":0
    ,"cdr":0
    ,"mmse":0
    ,"moca":0
    ,"alzheimer":0
    ,'cognitive impairment':0
    , "cognitiveimpairment":0
    ,"mci":0
    ,"cerebellar":0
    ,"neurocognitive":0
    ,"lewy":0
    ,"pick's":0
    ,"cortical":0
    ,"corticobasal":0
    ,"cerebral":0
    ,"cerebrovascular":0
    ,"amnesia":0
}

for k, v in match_tuples:    
    if (k != ''):
        freq.update({str(k.lower()):int(freq[k.lower()] + 1)})

#print(freq, i, "\n")

summary_stats["encounters_with_keywords"][436] = len(df5_matches[df5_matches["PatientID"] == unique_ids[436]])

for k, v in freq.items():
    summary_stats[str(k.lower())][i] = int(v)
    #print(k, v)

In [147]:
summary_stats

,patient_id,encounters_with_keywords,dementia,cognition,'cognition,memory,memory',cdr,mmse,moca,...,mci,cerebellar,neurocognitive,lewy,pick's,cortical,corticobasal,cerebral,cerebrovascular,amnesia
0,Z6352398,51.0,1.0,13.0,0.0,13.0,0.0,0.0,0.0,5.0,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0
1,Z6352614,18.0,0.0,9.0,0.0,2.0,0.0,0.0,0.0,0.0,...,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Z6352700,24.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,8.0,0.0,0.0
3,Z6352922,14.0,0.0,3.0,0.0,9.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
4,Z6352946,16.0,0.0,4.0,0.0,8.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2885,Z8712370,18.0,11.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2886,Z8713474,31.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
2887,Z8713489,22.0,0.0,16.0,0.0,1.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2888,Z8713893,95.0,1.0,23.0,0.0,47.0,0.0,0.0,1.0,0.0,...,1.0,9.0,1.0,0.0,0.0,2.0,0.0,8.0,0.0,0.0


In [148]:
summary_stats.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\1\apoe_1_summary_stats_updated.csv")

In [149]:
s1 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\1\apoe_1_summary_stats_updated.csv")
s2 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\2\apoe_2_summary_stats_updated.csv")
s3 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\3\apoe_3_summary_stats_updated.csv")
s4 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\4\apoe_4_summary_stats_updated.csv")
s5 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\5\apoe_5_summary_stats_updated.csv")

In [150]:
frames = [s1,s2,s3,s4,s5]
total_summary_stats = pd.concat(frames)

In [158]:
total_summary_stats['memory\''].value_counts()

0.0    2890
Name: memory', dtype: int64

In [160]:
total_summary_stats.head()

,'cognition,Unnamed: 0,alzheimer,amnesia,cdr,cerebellar,cerebral,cerebrovascular,cognition,cognitive impairment,...,encounters_with_keywords,lewy,mci,memory,memory',mmse,moca,neurocognitive,patient_id,pick's
0,0.0,0,0.0,0.0,0.0,0.0,12.0,0.0,13.0,4.0,...,51.0,0.0,3.0,13.0,0.0,0.0,5.0,0.0,Z6352398,0.0
1,0.0,1,0.0,0.0,0.0,2.0,0.0,0.0,9.0,0.0,...,18.0,0.0,5.0,2.0,0.0,0.0,0.0,0.0,Z6352614,0.0
2,0.0,2,0.0,0.0,0.0,1.0,8.0,0.0,13.0,0.0,...,24.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Z6352700,0.0
3,0.0,3,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,14.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,Z6352922,0.0
4,0.0,4,0.0,0.0,0.0,2.0,1.0,0.0,4.0,0.0,...,16.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,Z6352946,0.0


In [164]:
total_summary_stats.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\total_summary_stats_updated_2.csv")

In [162]:
s1[s1['patient_id'] == 'Z6356278']

,Unnamed: 0,patient_id,encounters_with_keywords,dementia,cognition,'cognition,memory,memory',cdr,mmse,...,mci,cerebellar,neurocognitive,lewy,pick's,cortical,corticobasal,cerebral,cerebrovascular,amnesia
27,27,Z6356278,1014.0,477.0,62.0,0.0,142.0,0.0,0.0,0.0,...,28.0,232.0,14.0,0.0,0.0,27.0,0.0,14.0,2.0,1.0


In [166]:
apoe_1_matches = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\1\apoe_1_matches.csv")

In [167]:
apoe_1_matches.head()

,Unnamed: 0,Unnamed: 0.1,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,NoteTXT,regex_location,regex_sent,regex_match,match_count
0,0,0,100000272,10040002098,Z6352398,3.076070e+09,2015-06-17 00:00:00.0000000,Office Visit,471433064,Progress Notes,1,453251959.0,1.0,----------------------------------------------...,"[(5850, 5858)]",'ncer. She passed secondary to this. He does n...,['cerebral'],1
1,1,1,100000272,10040002098,Z6352398,3.076214e+09,2015-07-16 00:00:00.0000000,Office Visit,513142600,Progress Notes,1,492483305.0,1.0,----------------------------------------------...,"[(3643, 3649)]",'a. Skin: No rashes or lesions. Neurological: ...,['memory'],1
2,2,2,100000272,10040002098,Z6352398,3.082899e+09,2015-10-15 00:00:00.0000000,Office Visit,640846456,Progress Notes,1,612720888.0,1.0,----------------------------------------------...,"[(6049, 6057)]",'ncer. She passed secondary to this. He does n...,['cerebral'],1
3,3,3,100000272,10040002098,Z6352398,3.085967e+09,2015-08-20 00:00:00.0000000,Office Visit,563095698,Progress Notes,1,539570111.0,1.0,----------------------------------------------...,"[(3810, 3816)]",'a. Skin: No rashes or lesions. Neurological: ...,['memory'],1
4,4,4,100000272,10040002098,Z6352398,3.094749e+09,2015-10-14 00:00:00.0000000,Abstract,638983522,Progress Notes,1,610990078.0,1.0,----------------------------------------------...,"[(5554, 5562)]",'ncer. She passed secondary to this. He does n...,['cerebral'],1


In [168]:
apoe_1_matches[apoe_1_matches['PatientID'] == 'Z6356278']

,Unnamed: 0,Unnamed: 0.1,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,NoteTXT,regex_location,regex_sent,regex_match,match_count
2145,2145,2145,100004841,10040037235,Z6356278,3.050473e+09,2012-03-28 00:00:00.0000000,System Generated,333918372,Discharge Summary,1,3.197316e+08,1.0,----------------------------------------------...,"[(6868, 6876)]","'012), CAD, thoracic ascending aortic aneurysm...",['dementia'],1
2146,2146,2146,100004841,10040037235,Z6356278,3.052879e+09,2009-01-24 00:00:00.0000000,System Generated,338298377,Discharge Summary,1,3.239686e+08,1.0,----------------------------------------------...,"[(2029, 2039), (2959, 2969), (4482, 4492), (93...",'us thrombosis. 5. Hypertension. 6. Hyperlipid...,['cerebellar'],1
2147,2147,2147,100004841,10040037235,Z6356278,3.052879e+09,2009-01-24 00:00:00.0000000,System Generated,338298377,Discharge Summary,1,3.239686e+08,1.0,----------------------------------------------...,"[(2029, 2039), (2959, 2969), (4482, 4492), (93...","assistance at home. Premorbidly, she was ambu...",['cerebellar'],1
2148,2148,2148,100004841,10040037235,Z6356278,3.052879e+09,2009-01-24 00:00:00.0000000,System Generated,338298377,Discharge Summary,1,3.239686e+08,1.0,----------------------------------------------...,"[(2029, 2039), (2959, 2969), (4482, 4492), (93...","deltoid abduction, which are 4/5 secondary to ...",['Cerebellar'],1
2149,2149,2149,100004841,10040037235,Z6356278,3.052879e+09,2009-01-24 00:00:00.0000000,System Generated,338298377,Discharge Summary,1,3.239686e+08,1.0,----------------------------------------------...,"[(2029, 2039), (2959, 2969), (4482, 4492), (93...",f daily living secondary to mechanical fall an...,['Cerebellar'],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3154,3154,3154,100004841,10040037235,Z6356278,3.301322e+09,2020-03-26 00:00:00.0000000,Hospital Encounter,3878883152,Discharge Summary,1,3.861220e+09,1.0,----------------------------------------------...,"[(1667, 1675), (2417, 2425), (2455, 2465), (40...","tion due to frequent falls, obstructive airway...","['dementia', 'cerebellar']",2
3155,3155,3155,100004841,10040037235,Z6356278,3.301322e+09,2020-03-26 00:00:00.0000000,Hospital Encounter,3878883152,Discharge Summary,1,3.861220e+09,1.0,----------------------------------------------...,"[(1667, 1675), (2417, 2425), (2455, 2465), (40...","e airway disease, GERD, diverticulosis, vascul...","['dementia', 'cerebellar']",2
3156,3156,3156,100004841,10040037235,Z6356278,3.301322e+09,2020-03-26 00:00:00.0000000,Hospital Encounter,3878883152,Discharge Summary,1,3.861220e+09,1.0,----------------------------------------------...,"[(1667, 1675), (2417, 2425), (2455, 2465), (40...",Mechanical fall with Closed fracture of lumba...,['cerebellar'],1
3157,3157,3157,100004841,10040037235,Z6356278,3.301322e+09,2020-03-26 00:00:00.0000000,Hospital Encounter,3878883152,Discharge Summary,1,3.861220e+09,1.0,----------------------------------------------...,"[(1667, 1675), (2417, 2425), (2455, 2465), (40...",al finding of spiculated right middle lobe nod...,['Dementia'],1


In [174]:
total = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\total_summary_stats_updated.csv")

In [175]:
total.columns

Index(['Unnamed: 0', 'patient_id', 'encounters_with_keywords', 'alzheimer',
       'amnesia', 'cdr', 'cerebellar', 'cerebral', 'cerebrovascular',
       'cognition', 'cognitive impairment', 'cognitiveimpairment', 'cortical',
       'corticobasal', 'dementia', 'lewy', 'mci', 'memory', 'mmse', 'moca',
       'pick's', 'neurocognitive'],
      dtype='object')

In [179]:
total[total['cognitiveimpairment'] == 8]

,Unnamed: 0,patient_id,encounters_with_keywords,alzheimer,amnesia,cdr,cerebellar,cerebral,cerebrovascular,cognition,...,cortical,corticobasal,dementia,lewy,mci,memory,mmse,moca,pick's,neurocognitive
6762,320.0,Z12259599,414.0,38.0,0.0,23.0,0.0,0.0,0.0,92.0,...,26.0,0.0,17.0,0.0,3.0,181.0,0.0,0.0,0.0,2.0
18046,NaN,NaN,1315273.0,53490.0,8399.0,5305.0,64670.0,117139.0,81830.0,264913.0,...,51337.0,274.0,107896.0,7181.0,176910.0,293194.0,3399.0,19180.0,49.0,18389.0
